# Export a BlockSync block model to OMF

This notebook shows how to download and export a BlockModel object from BlockSync to an OMF v1 file.

In this first cell we create a ServiceManagerWidget which will open a browser window and ask you to login.

Once logged in, a widget will be displayed below allowing you to select an organisation and workspace to export objects from.

__Required:__ You must fill in your `client_id` before running this cell.

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(client_id="your-client-id").login()

In the cell below we specify the Object UUID of the block model object we want to export and the output file path, and then call `export_blocksync_omf()`.

__Required:__ You must fill in your `object_id` of the BlockSync Object to export before running this cell.

__Note:__ At this stage only Regular block model types are supported.

In [ ]:
import os
from uuid import UUID

from evo.data_converters.omf.exporter import export_blocksync_omf

object_id = ""
version_id = None

output_dir = "data/output"
os.makedirs(output_dir, exist_ok=True)

output_file = f"{output_dir}/{object_id}.omf"

export_blocksync_omf(
    filepath=output_file,
    object_id=UUID(object_id),
    version_id=version_id,
    service_manager_widget=manager,
)

print(f"File saved to {output_file}")

### Optional: Download parquet file only

Use the cell below if you would like to inspect the parquet file associated with the blockmodel.

In [ ]:
import shutil

import pyarrow.parquet as pq

from evo.data_converters.common import BlockSyncClient, create_evo_object_service_and_data_client

service_client, data_client = create_evo_object_service_and_data_client(service_manager_widget=manager)
client = BlockSyncClient(service_client._environment, service_client._connector)

object_id = ""
dest_file = f"data/output/{object_id}.parquet"

job_url = client.get_blockmodel_columns_job_url(object_id)
download_url = client.get_blockmodel_columns_download_url(job_url)
downloaded_file = client.download_parquet(download_url)

shutil.copy(downloaded_file.name, dest_file)

table = pq.read_table(dest_file)

for column in table.column_names:
    print(f"{column} is of type: {table.schema.field(column).type}")